In [1]:
# only Sheet that is  Updating for Nepali Cars sheet
# updated and splitted title column to brandname, year and model
import pandas as pd

# df = pd.read_csv('output_data_final.csv');
sheet_name_or_index = 1
df = pd.read_excel('v1-output-data-updated-column-nepalicars.xlsx',sheet_name='Nepali Cars Data')
brandName = []
for index, eachRow in df.iterrows():
    title=eachRow['title']
    brandName=(title.split()[1])
    year=(title.split()[0])
    model = ' '.join(title.split()[2:4]) if len(title.split()) > 2 else ''
    df.at[index, 'brandName'] = brandName
    df.at[index, 'Make Year'] = year
    df.at[index, 'Model'] = model
    df.to_excel('update_nepali_cars_column.xlsx', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'v1-output-data-updated-column-nepalicars.xlsx'

In [ ]:
#To Copy all the remaining sheets along with the data to update for nepali cars
# updated and splitted title column to brandname, year and model
#  Also copied all the primary sheets remaining

import pandas as pd
from xlsxwriter import Workbook

dfs = pd.read_excel('output_data_final.xlsx', sheet_name=None)

# Specify the sheet you want to update
updating_sheet = 'Nepali Cars Data'

# Extract the DataFrame for the specified sheet
df = dfs[updating_sheet]
brandName = []
for index, eachRow in df.iterrows():
    title = eachRow['title']
    brandName = title.split()[1]
    year = title.split()[0]
    model = ' '.join(title.split()[2:4]) if len(title.split()) > 2 else ''
    df.at[index, 'brandName'] = brandName
    df.at[index, 'Make Year'] = year
    df.at[index, 'Model'] = model

# Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v1-output-data-updated-column-nepalicars.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df_sheet in dfs.items():
        if sheet_name == updating_sheet:
            # Save the updated DataFrame for the specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
#To Copy all the remaining sheets along with the data to update for Hamrobazar cars
# updated and splitted Transmission column to Drive(4WD, 2Wd), Gear(automatic/manual)
#  Also copied all the primary sheets remaining
import re
import pandas as pd
from xlsxwriter import Workbook

dfs = pd.read_excel('v1-output-data-updated-column-nepalicars.xlsx', sheet_name=None)

# Specify the sheet you want to update
updating_sheet = 'Hamro bazar data'

# Extract the DataFrame for the specified sheet
df = dfs[updating_sheet].copy() 
brandName = []
# for index, eachRow in df.iterrows():

drive_pattern = re.compile(r'(2WD|4WD|4 WD|2 WD)', flags=re.IGNORECASE)
df['DRIVE'] = (df['Transmission'].apply(lambda x: drive_pattern.search(str(x)) and drive_pattern.search(str(x)).group(0))).replace(" ", "")
# df['Drive'] = df['Transmission'].apply(lambda x: '2WD' if pd.notna(x) and ('2WD' in str(x).upper() or '2 WD' in str(x).upper()) else '4WD' if pd.notna(x) and ('4WD' in str(x).upper() or '4 WD' in str(x).upper()) else None)
automatic_pattern = re.compile(r'(Auto|Automatic|Power\s*Steering|PowerSteering)', flags=re.IGNORECASE)
manual_pattern = re.compile(r'Manual', flags=re.IGNORECASE)

# Apply regex patterns to create the 'Gear' column
df['GEARBOX']  = df['Transmission'].apply(lambda x: 'Automatic' if pd.notna(x) and automatic_pattern.search(str(x)) else 'Manual' if pd.notna(x) and manual_pattern.search(str(x)) else '-')

# df['Gear'] = df['Transmission'].apply(lambda x: 'Automatic' if pd.notna(x) and 'Auto' in str(x) or 'Power Steering' in str(x) else 'Manual')

# df.at[index, 'Drive'] = df['Drive'].fillna('-') 
display(df[['Transmission', 'GEARBOX', 'DRIVE']])




#Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v2-data-Transmission-drive-updated-column-hamrobazar.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df_sheet in dfs.items():
        if sheet_name == updating_sheet:
            # Save the updated DataFrame for the specified sheet
            df_combined = pd.concat([df_sheet, df[[ 'GEARBOX', 'DRIVE']]], axis=1)
            df_combined.to_excel(writer, sheet_name=sheet_name, index=False)

        else:
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)



In [ ]:
#To Copy all the remaining sheets along with the data to update for Nepali Cars
# updated and splitted Body Type column to Drive(4WD, 2Wd), Body Type(Hatchback/SUV)
# updated the date time human readbale format to Y-m-d type format
#  Also copied all the primary sheets remaining
import re
import pandas as pd
from xlsxwriter import Workbook
from dateutil.relativedelta import relativedelta
from datetime import datetime

def convert_to_date(date_str):
    try:
        # Use regular expressions to extract numerical values and units
        match = re.search(r'(\d+)\s+(\w+)', date_str)

        if match:
            value, unit = int(match.group(1)), match.group(2)
            delta = None

            if 'month' in unit:
                delta = relativedelta(months=value)
            elif 'year' in unit:
                delta = relativedelta(years=value)
            elif 'day' in unit:
                delta = relativedelta(days=value)

            if delta:
                return datetime.now() - delta

    except (ValueError, OverflowError):
        pass  # Handle parsing errors here

    return None
dfs = pd.read_excel('v2-hamrobazar-data-updated-transmission-to-gearbox-drive.xlsx', sheet_name=None)

# Specify the sheet you want to update
updating_sheet = 'Nepali Cars Data'

# Extract the DataFrame for the specified sheet
df = dfs[updating_sheet].copy() 
brandName = []
# for index, eachRow in df.iterrows():


#for drive(4WD and 2WD)
drive_pattern = re.compile(r'4 Wheel Drives', flags=re.IGNORECASE)
df['GEARBOX']  = df['BODY TYPE'].apply(lambda x: '4WD' if pd.notna(x) and drive_pattern.search(str(x)) else '2WD')

#for Body Type(SUV and Hatchback)
SUV_pattern = re.compile(r'4 Wheel Drives & SUVs', flags=re.IGNORECASE);
HatchBack_pattern=re.compile(r'Hatchback & Station Wagons', flags=re.IGNORECASE);
# Apply regex patterns to create the 'Wheel Drive' column
df['BODY'] = df['BODY TYPE'].apply(lambda x: 'SUV' if pd.notna(x) and SUV_pattern.search(str(x)) else 'Hatchback' if pd.notna(x) and HatchBack_pattern.search(str(x)) else x)
df['DATE'] = df['POSTDATE'].apply(convert_to_date)
display(df[['BODY TYPE', 'GEARBOX','BODY','DATE']])




# Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v3-nepalicars-data-updated-body-type-to-gear-box-&-body.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df_sheet in dfs.items():
        if sheet_name == updating_sheet:
            # Save the updated DataFrame for the specified sheet
            df_combined = pd.concat([df_sheet, df[[ 'GEARBOX', 'BODY','DATE']]], axis=1)
            df_combined.to_excel(writer, sheet_name=sheet_name, index=False)

        else:
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)
            

In [ ]:
#Merging nepali cars and Hamrobazar data into one sheet
import pandas as pd
from xlsxwriter import Workbook
excel_file_path = 'v3-nepalicars-data-updated-body-type-to-gear-box-&-body.xlsx';
df = pd.read_excel(excel_file_path, sheet_name=None)

df1 = pd.read_excel(excel_file_path, sheet_name='Hamro bazar data')

# Read the second sheet into another DataFrame
df2 = pd.read_excel(excel_file_path, sheet_name='Nepali Cars Data')

print("DataFrame 1 - Columns:", df1.columns)
print("DataFrame 1 - Index Name:", df1.index.name)

print("\nDataFrame 2 - Columns:", df2.columns)
print("DataFrame 2 - Index Name:", df2.index.name)


df1_subset = df1[['PRICE', 'BRANDNAME','MAKEYEAR','FUEL','KM','BODY','GEARBOX','DRIVE','COLOR','POSTDATE','MILEAGE','ENGINE(CC)']]
df2_subset = df2[['PRICE', 'BRANDNAME','MAKEYEAR','FUEL','KM','BODY','GEARBOX','DRIVE','COLOR','POSTDATE','MILEAGE','ENGINE(CC)']]
df1_subset.reset_index(drop=True, inplace=True)
df2_subset.reset_index(drop=True, inplace=True)
concatenated_df = pd.concat([df1_subset, df2_subset], ignore_index=True)
print(concatenated_df)

concatenated_df.index = range(1, len(concatenated_df) + 1)

# Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v4-hamrobazar-nepalicars-all-data-merged.xlsx', engine='openpyxl') as writer:
    concatenated_df.to_excel(writer, sheet_name='All Data', index=True ,startrow=1)
    for sheet_name, df_sheet in df.items():
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
df1.head()

In [ ]:
for k,v in df1:
    print(k)